## Longest Common Subsequence

> **QUESTION 1**: Write a function to find the length of the **longest common subsequence** between two sequences. E.g. Given the strings "serendipitous" and "precipitation", the longest common subsequence is "reipito" and its length is 7.
>
> A "sequence" is a group of items with a deterministic ordering. Lists, tuples and ranges are some common sequence types in Python.
>
> A "subsequence" is a sequence obtained by deleting zero or more elements from another sequence. For example, "edpt" is a subsequence of "serendipitous".




#### General case

<img src="https://i.imgur.com/ry4Y0wS.png" width="420">

#### Test cases

1. General case (string)
2. General case (list)
3. No common subsequence
4. One is a subsequence of the other
5. One sequence is empty
6. Both sequences are empty
7. Multiple subsequences with same length
    1. “abcdef” and “badcfe”



Longest common subsequence test cases:

In [ ]:
T0 = {
    'input': {
        'seq1': 'serendipitous',
        'seq2': 'precipitation'
    },
    'output': 7
}

T1 = {
    'input': {
        'seq1': [1, 3, 5, 6, 7, 2, 5, 2, 3],
        'seq2': [6, 2, 4, 7, 1, 5, 6, 2, 3]
    },
    'output': 5
}

T2 = {
    'input': {
        'seq1': 'longest',
        'seq2': 'stone'
    },
    'output': 3
}

T3 = {
    'input': {
        'seq1': 'asdfwevad',
        'seq2': 'opkpoiklklj'
    },
    'output': 0
}

T4 = {
    'input': {
        'seq1': 'dense',
        'seq2': 'condensed'
    },
    'output': 5
}

T5 = {
    'input': {
        'seq1': '',
        'seq2': 'opkpoiklklj'
    },
    'output': 0
}

T6 = {
    'input': {
        'seq1': '',
        'seq2': ''
    },
    'output': 0
}

T7 = {
    'input': {
        'seq1': 'abcdef',
        'seq2': 'badcfe'
    },
    'output': 3
}

In [ ]:
lcq_tests = [T0, T1, T2, T3, T4, T5, T6, T7]

#### Recursive Solution


1. Create two counters `idx1` and `idx2` starting at 0. Our recursive function will compute the LCS of `seq1[idx1:]` and `seq2[idx2:]`


2. If `seq1[idx1]` and `seq2[idx2]` are equal, then this character belongs to the LCS of `seq1[idx1:]` and `seq2[idx2:]` (why?). Further the length this is LCS is one more than LCS of `seq1[idx1+1:]` and  `seq2[idx2+1:]`

<img src="https://i.imgur.com/um7LDiX.png" width="400">

3. If not, then the LCS of `seq1[idx1:]` and `seq2[idx2:]` is the longer one among the LCS of `seq1[idx1+1:], seq2[idx2:]` and the LCS of `seq1[idx1:]`, `seq2[idx2+1:]`

<img src="https://i.imgur.com/DRanmOy.png" width="360">

5. If either `seq1[idx1:]` or `seq2[idx2:]` is empty, then their LCS is empty.



Here's what the tree of recursive calls looks like:


![](https://i.imgur.com/JJrq3KH.png)





In [ ]:
def lsc_recursive(seq1, seq2, idx1=0, idx2=0):
    if len(seq1[idx1:]) == 0 or len(seq2[idx2:]) == 0:
        return 0
    elif seq1[idx1] == seq2[idx2]:
        result = 1 + lsc_recursive(seq1, seq2, idx1 + 1, idx2 + 1)
    else:
        result = max(lsc_recursive(seq1, seq2, idx1 + 1, idx2), lsc_recursive(seq1, seq2, idx1, idx2 + 1))

    return result

In [ ]:
import jovian

In [ ]:
from jovian.pythondsa import evaluate_test_cases

In [ ]:
evaluate_test_cases(lsc_recursive, lcq_tests)

In [ ]:
def lsc_recursive_memo(seq1, seq2):
    
    memo = {}
    
    def recurse(idx1=0, idx2=0):
        key = (idx1, idx2)
        if key in memo:
            return memo[key]      
        elif len(seq1[idx1:]) == 0 or len(seq2[idx2:]) == 0:
            memo[key] = 0
        elif seq1[idx1] == seq2[idx2]:
            memo[key] = 1 + recurse(idx1 + 1, idx2 + 1)
        else:
            memo[key] = max(recurse(idx1 + 1, idx2), recurse(idx1, idx2 + 1))

        return memo[key]
    
    return recurse(0, 0)

In [ ]:
evaluate_test_cases(lsc_recursive_memo, lcq_tests)

#### Complexity Analysis

Worst case occurs when each time we have to try 2 subproblems i.e. when the sequences have no common elements.

<img src="https://i.imgur.com/z5m36m8.png" width="360">

Here's what the tree looks like in such a case (source - Techie Delight):

<img src="https://i.imgur.com/n8ZgBYj.png" width="500">

All the leaf nodes are `(0, 0)`. Can you count the number of leaf nodes?

*HINT*: Count the number of unique paths from root to leaf. The length of each path is `m+n` and at each level there are 2 choices. 

Based on the above can you infer that the time complexity is $O(2^{m+n})$.









### Dynamic programming

1. Create a table of size `(n1+1) * (n2+1)` initialized with 0s, where `n1` and `n2` are the lengths of the sequences. `table[i][j]` represents the longest common subsequence of `seq1[:i]` and `seq2[:j]`. Here's what the table looks like (source: Kevin Mavani, Medium).


<img src="https://i.imgur.com/SAsEol6.png">



2. If `seq1[i]` and `seq2[j]` are equal, then `table[i+1][j+1] = 1 + table[i][j]` 

3. If `seq1[i]` and `seq2[j]` are equal, then `table[i+1][j+1] = max(table[i][j+1], table[i+1][j])`


Verify that the complexity of the dynamic programming approach is $O(N1 * N2)$.

In [ ]:
def lcs_dynamic(seq1, seq2):
    result_table = [[0 for i in range(len(seq2) + 1)] for j in range(len(seq1) + 1)]
    #print(result_table)
    for i in range(len(seq1)):
        for j in range(len(seq2)):
            if seq1[i] == seq2[j]:
                result_table[i+1][j+1] = 1 + result_table[i][j]
            else:
                result_table[i+1][j+1] = max(result_table[i][j+1], result_table[i+1][j])

    return result_table[-1][-1]

In [ ]:
lcs_dynamic('stone', 'longest')

In [ ]:
evaluate_test_cases(lcs_dynamic, lcq_tests)

## 0-1 Knapsack Problem

#### Problem statement

> You’re in charge of selecting a football (soccer) team from a large pool of players. Each player has a cost, and a rating. You have a limited budget. What is the highest total rating of a team that fits within your budget. Assume that there’s no minimum or maximum team size.



General problem statemnt:

> Given n elements, each of which has a weight and a profit, determine the maximum profit that can be obtained by selecting a subset of the elements weighing no more than w.

<img src="https://i.imgur.com/4O919vu.png" width="400">


Test cases:

1. Some generic test cases
2. All the elements can be included
3. None of the elements can be included
4. Only one of the elements can be included
5. ???




Knapsack test cases:

In [ ]:
test0 = {
    'input': {
        'capacity': 165,
        'weights': [23, 31, 29, 44, 53, 38, 63, 85, 89, 82],
        'profits': [92, 57, 49, 68, 60, 43, 67, 84, 87, 72]
    },
    'output': 309
}

test1 = {
    'input': {
        'capacity': 3,
        'weights': [4, 5, 6],
        'profits': [1, 2, 3]
    },
    'output': 0
}

test2 = {
    'input': {
        'capacity': 4,
        'weights': [4, 5, 1],
        'profits': [1, 2, 3]
    },
    'output': 3
}

test3 = {
    'input': {
        'capacity': 170,
        'weights': [41, 50, 49, 59, 55, 57, 60],
        'profits': [442, 525, 511, 593, 546, 564, 617]
    },
    'output': 1735
}

test4 = {
    'input': {
        'capacity': 15,
        'weights': [4, 5, 6],
        'profits': [1, 2, 3]
    },
    'output': 6
}

test5 = {
    'input': {
        'capacity': 15,
        'weights': [4, 5, 1, 3, 2, 5],
        'profits': [2, 3, 1, 5, 4, 7]
    },
    'output': 19
}

In [ ]:
tests = [test0, test1, test2, test3, test4, test5]

#### Recursion

<img src="https://i.imgur.com/eaJzv02.png" width="400">

1. We'll write a recursive function that computes `max_profit(weights[idx:], profits[idx:], capacity)`, with `idx` starting from 0.


2. If `weights[idx] > capacity`, the current element is cannot be selected, so the maximum profit is the same as `max_profit(weights[idx+1:], profits[idx+1:], capacity)`.


3. Otherwise, there are two possibilities: we either pick `weights[idx]` or don't. We can recursively compute the maximum

    A. If we don't pick `weights[idx]`, once again the maximum profit for this case is `max_profit(weights[idx+1:], profits[idx+1:], capacity)`

    B. If we pick `weights[idx]`, the maximum profit for this case is `profits[idx] + max_profit(weights[idx+1:], profits[idx+1:], capacity - weights[idx]`


4. If `weights[idx:]` is empty, the maximum profit for this case is 0.





Here's a visualization of the recursion tree:

<img src="https://i.imgur.com/WsKTC6I.png" width="640">


Verify that the time complexity of the recursive algorithm is $O(2^N)$


In [ ]:
def knapsack_recursive(weights, profits, capacity):
    pass

In [ ]:
evaluate_test_cases(knapsack_recursive, tests)

In [ ]:
def knapsack_recursive(weights, profits, capacity, idx=0):
    if idx == len(weights):
        return 0
    elif weights[idx] > capacity:
        return knapsack_recursive(weights, profits, capacity, idx +1)
    else:
        option1 = knapsack_recursive(weights, profits, capacity, idx +1)
        option2 = profits[idx] + knapsack_recursive(weights, profits, capacity - weights[idx], idx +1)
        return max(option1, option2)

In [ ]:
evaluate_test_cases(knapsack_recursive, tests)

In [ ]:
def knapsack_recursive_memo(weights, profits, capacity):

    memo = {}

    def recurse(idx = 0):
        key = idx

        if key in memo:
            return memo[key]
        elif idx == len(weights):
            memo[key] = 0
        elif weights[idx] > capacity:
            memo[key] = knapsack_recursive(weights, profits, capacity, idx +1)
        else:
            option1 = knapsack_recursive(weights, profits, capacity, idx +1)
            option2 = profits[idx] + knapsack_recursive(weights, profits, capacity - weights[idx], idx +1)
            memo[key] = max(option1, option2)
            
        return memo[key]
        
    return recurse(0)

In [ ]:
evaluate_test_cases(knapsack_recursive_memo, tests)

#### Dynamic Programming

1. Create a table of size `(n+1) * (capacity+1)` consisting of all 0s, where is `n` is the number of elements. `table[i][c]` represents the maximum profit that can be obtained using the first `i` elements if the maximum capacity is `c`. Here's a visual representation of a filled table (source - geeksforgeeks):

<img src="https://i2.wp.com/techieme.in/wp-content/uploads/01knapsack.png?w=1213" width="640">

(The 0th row will contain all zeros and is not shown above.)

2. We'll fill the table row by row and column by column. `table[i][c]` can be filled using some values in the row above it.

3. If `weights[i] > c` i.e. if the current element can is larger than capacity, then `table[i+1][c]` is simply equal to `table[i][c]` (since there's no way we can pick this element).

4. If `weights[i] <= c` then we have two choices: to either pick the current element or not to get the value of `table[i+1][c]`. We can compare the maximum profit for both these options and pick the better one as the value of `table[i][c]`.

    A. If we don't pick  the element with weight `weights[i]`, then once again the maximum profit is `table[i][c]`
    
    B. If we pick the element with weight `weights[i]`, then the maximum profit is `profits[i] + table[i][c-weights[i]]`, since we have used up some capacity.
    


Verify that the complexity of the dynamic programming solution is $O(N * W)$.

In [ ]:
def knapsack_dynamic(weights, profits, capacity):
    n1, n2 = len(weights), capacity
    result_table = [[0 for _ in range(n2 + 1)] for _ in range(n1+ 1)]
    

    for i in range(n1):
        for j in range(n2+1):
            if weights[i] > j:
                result_table[i+1][j] = result_table[i][j]
            else:
                result_table[i+1][j] = max(result_table[i][j], profits[i] + result_table[i][j-weights[i]])
    
    return result_table[-1][-1]

In [ ]:
knapsack_dynamic([4, 5, 1, 2], [1, 2, 3, 4], 5)

In [ ]:
evaluate_test_cases(knapsack_dynamic, tests)

Longest common subsequence:

In [ ]:
def lcq_recursive(seq1, seq2, idx1=0, idx2=0):
    # Check if either of the sequences is exhausted
    if idx1 == len(seq1) or idx2 == len(seq2):
        return 0
    
    # Check if the current characters are equal
    if seq1[idx1] == seq2[idx2]:
        return 1 + lcq_recursive(seq1, seq2, idx1+1, idx2+1)
    # Skip one element from each sequence
    else:
        return max(lcq_recursive(seq1, seq2, idx1+1, idx2), 
                   lcq_recursive(seq1, seq2, idx1, idx2+1))    

In [ ]:
from jovian.pythondsa import evaluate_test_cases

In [ ]:
evaluate_test_cases(lcq_recursive, lcq_tests)

In [ ]:
%%time
lcq_recursive('seredipitous', 'precipitation')

In [ ]:
%%time
lcq_recursive('Asdfsfafssess', 'oypiououuiuo')

In [ ]:
def lcq_memoized(seq1, seq2):
    memo = {}
    
    def recurse(idx1, idx2):
        key = idx1, idx2
        
        if key in memo:
            return memo[key]
        
        if idx1 == len(seq1) or idx2 == len(seq2):
            memo[key] = 0
        elif seq1[idx1] == seq2[idx2]:
            memo[key] = 1 + recurse(idx1+1, idx2+1)
        else:
            memo[key] = max(recurse(idx1+1, idx2), 
                            recurse(idx1, idx2+1))
        return memo[key]
        
    return recurse(0, 0)

In [ ]:
evaluate_test_cases(lcq_memoized, lcq_tests)

In [ ]:
%%time
lcq_memoized('Asdfsfafssess', 'oypiououuiuo')

In [ ]:
%%time
lcq_memoized('seredipitous', 'precipitation')

In [ ]:
%%time
lcq_memoized('longest', 'stone')

Dynamic programming:

In [ ]:
def lcq_dp(seq1, seq2):
    n1, n2 = len(seq1), len(seq2)
    results = [[0 for _ in range(n2+1)] for _ in range(n1+1)]
    for idx1 in range(n1):
        for idx2 in range(n2):
            if seq1[idx1] == seq2[idx2]:
                results[idx1+1][idx2+1] = 1 + results[idx1][idx2]
            else:
                results[idx1+1][idx2+1] = max(results[idx1][idx2+1], results[idx1+1][idx2])
    return results[-1][-1]

In [ ]:
evaluate_test_cases(lcq_dp, lcq_tests)

In [ ]:
%%time
lcq_dp('Asdfsfafssess', 'oypiououuiuo')

In [ ]:
%%time
lcq_dp('seredipitous', 'precipitation')

In [ ]:
%%time
lcq_dp('longest', 'stone')

### Knapsack Solutions

In [ ]:
from jovian.pythondsa import evaluate_test_cases

In [ ]:
def max_profit_recursive(capacity, weights, profits, idx=0):
    if idx == len(weights):
        return 0
    if weights[idx] > capacity:
        return max_profit_recursive(capacity, weights, profits, idx+1)
    else:
        return max(max_profit_recursive(capacity, weights, profits, idx+1),
                   profits[idx] + max_profit_recursive(capacity-weights[idx], weights, profits, idx+1))

In [ ]:
evaluate_test_cases(max_profit_recursive, tests)

Memoized:

In [ ]:
def knapsack_memo(capacity, weights, profits):
    memo = {}
    
    def recurse(idx, remaining):
        key = (idx, remaining)
        if key in memo:
            return memo[key]
        elif idx == len(weights):
            memo[key] = 0
        elif weights[idx] > remaining:
            memo[key] = recurse(idx+1, remaining)
        else:
            memo[key] = max(recurse(idx+1, remaining), 
                            profits[idx] + recurse(idx+1, remaining-weights[idx]))
        return memo[key] 
        
    return recurse(0, capacity)

In [ ]:
evaluate_test_cases(knapsack_memo, tests)

Dynamic programming:

In [ ]:
def knapsack_dp(capacity, weights, profits):
    n = len(weights)
    results = [[0 for _ in range(capacity+1)] for _ in range(n+1)]
    
    for idx in range(n):
        for c in range(capacity+1):
            if weights[idx] > c:
                results[idx+1][c] = results[idx][c]
            else:
                results[idx+1][c] = max(results[idx][c], profits[idx] + results[idx][c-weights[idx]])
            
    return results[-1][-1]

In [ ]:
evaluate_test_cases(knapsack_dp, tests)

In [72]:
import jovian

In [73]:
jovian.commit(filename="dynamic-programming-problems")

<IPython.core.display.Javascript object>

[jovian] Error: Looks like the notebook is missing output cells, please save the notebook and try jovian.commit again.


[jovian] Committed successfully! https://jovian.com/triple-espress0/dynamic-programming-problems-3072b


'https://jovian.com/triple-espress0/dynamic-programming-problems-3072b'